# Atomate2 introduction

Atomate2 is an open-source library providing computational workflows for automating first-principles and machine-learned based calculations.
Supported DFT codes include

- ABINIT
- CP2K
- FHI-AIMS
- JDFTx
- Q-Chem
- VASP

but in this example only MLFF will be considered, as they are fast to execute and need less set up installation.
  

## MLIP based workflows

Workflows using MLIPs like MACE, CHGNET, M3GNET, GAP, NEP, SevenNet are available for:

- relaxation
- molecular dynamics
- equation of state
- elastic tensor
- phonons and Grüneisen parameter computations using [phonopy](https://github.com/phonopy/phonopy)

It is easy to customise and compose any of the above workflows.

# Installation and Setup 

Typically, before using workflows available in atomate2, you will need to create a Python environment with the necessary packages and set some configuration files where you specify details like a path to VASP binary and your database information like collection names, usernames, passwords, etc. You can find this in the [atomate2 documentation](https://materialsproject.github.io/atomate2/user/install.html).


This has already been set up in the Docker container, so we can directly create workflows and submit them to the local or slurm worker. We will use [jobflow-remote](https://github.com/Matgenix/jobflow-remote) to manage and execute workflows. However, you can also use [Fireworks](https://materialsproject.github.io/atomate2/user/fireworks.html).

# Workflows

A Workflow is basically a sequence of tasks where the connectivity, execution order, and dependencies of the different subtasks are predefined using `Flow` or `job` objects. For writing the inputs and parsing (reading) the outputs of calculations, atomate2 mainly relies on [pymatgen](https://github.com/materialsproject/pymatgen) and [ase](https://gitlab.com/ase/ase) packages. Thus it might be beneficial to get familiar with these packages. 

The output of a typical workflow in `atomate2` consists of a summary of the most relevant information for the specific calculation in a JSON serializable dict and is commonly referred to as a `TaskDocument`. The name of this `TaskDocument` and information contained within changes depending on the type of calculation and underlying calculator (e.g.:- DFT code, MLIP ) used in the Workflow.

In this session, we will start from very simple workflows and gradually increase the complexity. 

## Running a  structure relaxation workflow

In the following example we will be using the [CHGNet](https://github.com/CederGroupHub/chgnet/) MLIP. But other MLIPs can be used as well.

In [ ]:
from pymatgen.core import Structure
from atomate2.forcefields.jobs import ForceFieldRelaxMaker
from jobflow_remote import submit_flow

In [ ]:
# construct a rock salt MgO structure
mgo_structure = Structure(
    lattice=[[0, 2.13, 2.13], [2.13, 0, 2.13], [2.13, 2.13, 0]],
    species=["Mg", "O"],
    coords=[[0, 0, 0], [0.5, 0.5, 0.5]],
)

#### Check the customizable parameters for initializing the workflow

In [ ]:
ForceFieldRelaxMaker?

In [ ]:
ForceFieldRelaxMaker.make?

In order to make a meaningful example for the relaxation we will randomly displace the atoms inside the box and scale the cell size.

In [ ]:
mgo_structure_rattled = mgo_structure.copy()
mgo_structure_rattled.perturb(0.1)
mgo_structure_rattled.scale_lattice(mgo_structure.volume*1.05)

In [ ]:
relax_job = ForceFieldRelaxMaker(force_field_name="CHGNet").make(structure=mgo_structure_rattled) # initialize the flow
relax_job.name = "Relax Job" # one can also set a custom name to flow 

In [ ]:
resources={"nodes": 1 , "ntasks": 1, "time": "01:00:00"} # define the resources for the jobs in flow
submit_flow(relax_job, worker="local_slurm", resources=resources) # set the resources for job execution

In [ ]:
! jf runner start # optional

In [ ]:
!jf flow list # overview of flows added to the DB

In [ ]:
!jf job list # Get list of jobs in the DB

In [ ]:
!jf job info 3

If you prefer to monitor jobs via a graphical user interface simply run. This should start a gui server which can be opened in a browser

```bash
jf gui
```

#### Access the relax workflow results from the database and download the optimized structure

For core MLIP jobs like the one here (Structure Relaxation), the output document can be found in this [atomate2 module
](https://github.com/materialsproject/atomate2/blob/main/src/atomate2/forcefields/schemas.py) and is named `ForceFieldTaskDocument`.

So now, we will access this document which is stored in the database after successful run of the workflow. To do that we will first fetch the output of the Job as a dictionary from the MongoDB database, then we will deserialize it, taking advantage of the standard (de)serialization tools used in the Material Project (i.e. the `MontyEncoder`/`MontyDecoder`).

In [ ]:
from jobflow_remote import get_jobstore
from monty.serialization import MontyDecoder
import matplotlib.pyplot as plt

In [ ]:
# connect to the database where the results are stored
jobstore = get_jobstore()
jobstore.connect()

In [ ]:
# get output
relax_doc = jobstore.get_output(uuid=relax_job.uuid, load=True)

# get python TaskDoc object from serialized output retrieved from DB
task_doc_relax = MontyDecoder().process_decoded(relax_doc)

In [ ]:
# Access VASP calculation inputs like incar, kpoints, poscar
task_doc_relax.input

In [ ]:
# Access final energy
task_doc_relax.output.energy

In [ ]:
# Access forces
task_doc_relax.output.forces

In [ ]:
# Access stress
task_doc_relax.output.stress

In [ ]:
# Access / Download final optimized structure
task_doc_relax.output.structure.to_file("MgO_opt.cif", fmt="cif")

In [ ]:
# Access directory where the calculation ran
task_doc_relax.dir_name

In [ ]:
# Collect energies for each optimization step from both relaxation tasks
energy_step = [
    step.energy for step in task_doc_relax.output.ionic_steps
]

# Generate step indices
opt_step = list(range(1, len(energy_step) + 1))

# Compute energy difference relative to the first step
del_energy_step = [e - energy_step[0] for e in energy_step]

# Plot results
plt.plot(opt_step, del_energy_step)
plt.ylabel(r'$\Delta E$')
plt.xlabel("Ionic step")
plt.show()

## Running a Phonon workflow with MLIP and accessing the results

In [ ]:
from pymatviz.structure_viz import structure_3d_plotly
from atomate2.forcefields.flows.phonons import PhononMaker
from atomate2.vasp.powerups import add_metadata_to_flow
from pymatgen.core import Structure
from jobflow_remote import submit_flow

In [ ]:
# Read si structure
si_structure = Structure.from_file("Si.vasp")

In [ ]:
structure_3d_plotly(si_structure)

In [ ]:
# make a Phonon flow to optimise the structure and obtain the Phonon bandstructure
# It is possible to further customize calculation parameters, but for this case we stick to defaults
phonon_flow = PhononMaker(
    min_length=8.0, # here we set small supercell length for faster calculations (ideally you would use 15-20 angstroms supercell)
    use_symmetrized_structure="conventional",
    create_thermal_displacements=False,
    store_force_constants=False,
    prefer_90_degrees=False,
    generate_frequencies_eigenvectors_kwargs={"tstep": 100, "band_structure_eigenvectors": False},
).make(si_structure)

phonon_flow.name = "Phonon flow"

In [ ]:
resources={"nodes": 1 , "ntasks": 1, "time": "01:00:00"} # define the resources for the jobs in flow

In [ ]:
submit_flow(phonon_flow, worker="local_slurm", resources=resources) # set the resources for job execution

In [ ]:
# get overview of the list of jobs added and its status
! jf job list

#### Access the phonon workflow results from the database and plot phonon bandstructure

Each of the atomate2 workflows mostly have a associated [Taskdocument](https://materialsproject.github.io/atomate2/user/key_concepts_overview.html) generated for specific type of calculations. The Taskdocument can be understood simply as summary of most important results from the calculation.

For phonon workflow, the taskdocument is named as [PhononBSDOSDoc](https://github.com/materialsproject/atomate2/blob/b8ff9f444bcecaac474f3779acd83ee3b5ad4ae1/src/atomate2/common/schemas/phonons.py#L136). 

So now, we will access this document which is stored in the database after successful run of the workflow.

In [ ]:
from jobflow_remote import get_jobstore
from monty.serialization import MontyDecoder
from pymatgen.phonon.bandstructure import PhononBandStructureSymmLine
from pymatgen.phonon.dos import PhononDos
from pymatgen.phonon.plotter import PhononBSPlotter, PhononDosPlotter

In [ ]:
# connect to the database where the results are stored
jobstore = get_jobstore()
jobstore.connect()

In [ ]:
# get output doc
phonon_doc = jobstore.get_output(phonon_flow.output.uuid, load=True)

# get python PhononBSDOSDoc object from serialized output retrieved from DB
task_doc_phonon = MontyDecoder().process_decoded(phonon_doc)

In [ ]:
# initialize dos plotter and visualize dos plot
dos_plot = PhononDosPlotter()
dos_plot.add_dos(label="Total Dos", dos=task_doc_phonon.phonon_dos)
dos_plot.get_plot();

# initialize Phonon bandstructure plotter and visualize band structure plot
bs_plot = PhononBSPlotter(bs=task_doc_phonon.phonon_bandstructure)
bs_plot.get_plot();

You can compare these results with those calculate with DFT available on the [Materials Project website](https://next-gen.materialsproject.org/materials/mp-149) (in the "Properties" section, under the "Phonon" tab)

# Optional 

- Run Elastic constant or a Equation of state workflow and access the results
- Try different force fields. For example replace `CHGNet` with `M3GNet` in the `force_field_name` option.